In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 32
tree_depth = 6
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.164664268493652 | KNN Loss: 6.227475166320801 | BCE Loss: 1.937188982963562
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.172765731811523 | KNN Loss: 6.227191925048828 | BCE Loss: 1.9455742835998535
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.190939903259277 | KNN Loss: 6.227177143096924 | BCE Loss: 1.9637631177902222
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.146702766418457 | KNN Loss: 6.226847171783447 | BCE Loss: 1.9198553562164307
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.167062759399414 | KNN Loss: 6.2267584800720215 | BCE Loss: 1.9403047561645508
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.134511947631836 | KNN Loss: 6.226141452789307 | BCE Loss: 1.908370018005371
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.144550323486328 | KNN Loss: 6.225564002990723 | BCE Loss: 1.9189860820770264
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.135579109191895 | KNN Loss: 6.225414752960205 | BCE Loss: 1.910163

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 5.511267185211182 | KNN Loss: 4.427506923675537 | BCE Loss: 1.0837602615356445
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 5.47360897064209 | KNN Loss: 4.334427833557129 | BCE Loss: 1.139181137084961
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 5.351345062255859 | KNN Loss: 4.25172233581543 | BCE Loss: 1.0996226072311401
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 5.377434730529785 | KNN Loss: 4.280810356140137 | BCE Loss: 1.0966246128082275
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 5.298580169677734 | KNN Loss: 4.186285972595215 | BCE Loss: 1.1122944355010986
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 5.240612030029297 | KNN Loss: 4.130659580230713 | BCE Loss: 1.1099525690078735
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 5.193686485290527 | KNN Loss: 4.103450775146484 | BCE Loss: 1.090235710144043
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 5.167356967926025 | KNN Loss: 4.063889980316162 | BCE Loss: 1.

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 4.867629528045654 | KNN Loss: 3.8126978874206543 | BCE Loss: 1.0549317598342896
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 4.894550800323486 | KNN Loss: 3.8383522033691406 | BCE Loss: 1.0561985969543457
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 4.868419647216797 | KNN Loss: 3.809732437133789 | BCE Loss: 1.058687448501587
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 4.86475944519043 | KNN Loss: 3.838819742202759 | BCE Loss: 1.0259398221969604
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 4.947162628173828 | KNN Loss: 3.889233350753784 | BCE Loss: 1.057929515838623
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 4.845092296600342 | KNN Loss: 3.7960262298583984 | BCE Loss: 1.0490660667419434
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 4.894075393676758 | KNN Loss: 3.827622175216675 | BCE Loss: 1.066453456878662
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 4.861667633056641 | KNN Loss: 3.7902114391326904 | BCE Los

Epoch 32 / 500 | iteration 10 / 30 | Total Loss: 4.831170558929443 | KNN Loss: 3.7563765048980713 | BCE Loss: 1.074794054031372
Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 4.797304153442383 | KNN Loss: 3.7578086853027344 | BCE Loss: 1.0394952297210693
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 4.778392791748047 | KNN Loss: 3.75398850440979 | BCE Loss: 1.0244042873382568
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 4.798393249511719 | KNN Loss: 3.75352144241333 | BCE Loss: 1.0448715686798096
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 4.7605390548706055 | KNN Loss: 3.7299633026123047 | BCE Loss: 1.0305759906768799
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 4.825061321258545 | KNN Loss: 3.779970407485962 | BCE Loss: 1.0450907945632935
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 4.787725448608398 | KNN Loss: 3.7464547157287598 | BCE Loss: 1.0412707328796387
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 4.800629615783691 | KNN Loss: 3.7373907566070557 | BCE 

Epoch 43 / 500 | iteration 5 / 30 | Total Loss: 4.838930130004883 | KNN Loss: 3.808166027069092 | BCE Loss: 1.0307643413543701
Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 4.802123069763184 | KNN Loss: 3.764037847518921 | BCE Loss: 1.0380849838256836
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 4.778770923614502 | KNN Loss: 3.7332231998443604 | BCE Loss: 1.0455477237701416
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 4.752194881439209 | KNN Loss: 3.7203030586242676 | BCE Loss: 1.0318917036056519
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 4.779771327972412 | KNN Loss: 3.7534127235412598 | BCE Loss: 1.0263586044311523
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 4.772917747497559 | KNN Loss: 3.726386070251465 | BCE Loss: 1.0465319156646729
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 4.778808116912842 | KNN Loss: 3.7498276233673096 | BCE Loss: 1.0289804935455322
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 4.833897590637207 | KNN Loss: 3.7689502239227295 | BCE

...

Epoch 448 / 500 | iteration 10 / 30 | Total Loss: 4.748598098754883 | KNN Loss: 3.703897476196289 | BCE Loss: 1.0447006225585938
Epoch 448 / 500 | iteration 15 / 30 | Total Loss: 4.709958553314209 | KNN Loss: 3.684103488922119 | BCE Loss: 1.0258550643920898
Epoch 448 / 500 | iteration 20 / 30 | Total Loss: 4.682947158813477 | KNN Loss: 3.665107488632202 | BCE Loss: 1.0178394317626953
Epoch 448 / 500 | iteration 25 / 30 | Total Loss: 4.696419715881348 | KNN Loss: 3.6882333755493164 | BCE Loss: 1.0081863403320312
Epoch   449: reducing learning rate of group 0 to 6.7053e-07.
Epoch 449 / 500 | iteration 0 / 30 | Total Loss: 4.688918113708496 | KNN Loss: 3.6989476680755615 | BCE Loss: 0.9899702668190002
Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 4.684876918792725 | KNN Loss: 3.6805100440979004 | BCE Loss: 1.0043667554855347
Epoch 449 / 500 | iteration 10 / 30 | Total Loss: 4.676281929016113 | KNN Loss: 3.67991304397583 | BCE Loss: 0.9963686466217041
Epoch 449 / 500 | iteration 15 / 30

Epoch 459 / 500 | iteration 0 / 30 | Total Loss: 4.739774703979492 | KNN Loss: 3.69266939163208 | BCE Loss: 1.047105073928833
Epoch 459 / 500 | iteration 5 / 30 | Total Loss: 4.714485168457031 | KNN Loss: 3.6632633209228516 | BCE Loss: 1.0512220859527588
Epoch 459 / 500 | iteration 10 / 30 | Total Loss: 4.720749855041504 | KNN Loss: 3.6967527866363525 | BCE Loss: 1.0239968299865723
Epoch 459 / 500 | iteration 15 / 30 | Total Loss: 4.785182476043701 | KNN Loss: 3.7344589233398438 | BCE Loss: 1.0507235527038574
Epoch 459 / 500 | iteration 20 / 30 | Total Loss: 4.728562355041504 | KNN Loss: 3.7009694576263428 | BCE Loss: 1.027592658996582
Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 4.679009437561035 | KNN Loss: 3.6754894256591797 | BCE Loss: 1.0035202503204346
Epoch   460: reducing learning rate of group 0 to 4.6937e-07.
Epoch 460 / 500 | iteration 0 / 30 | Total Loss: 4.720353603363037 | KNN Loss: 3.7050390243530273 | BCE Loss: 1.0153146982192993
Epoch 460 / 500 | iteration 5 / 30 

Epoch 469 / 500 | iteration 20 / 30 | Total Loss: 4.723755836486816 | KNN Loss: 3.6922805309295654 | BCE Loss: 1.031475305557251
Epoch 469 / 500 | iteration 25 / 30 | Total Loss: 4.755258083343506 | KNN Loss: 3.71797251701355 | BCE Loss: 1.037285566329956
Epoch 470 / 500 | iteration 0 / 30 | Total Loss: 4.722499370574951 | KNN Loss: 3.7153282165527344 | BCE Loss: 1.0071711540222168
Epoch 470 / 500 | iteration 5 / 30 | Total Loss: 4.697483062744141 | KNN Loss: 3.651270627975464 | BCE Loss: 1.0462125539779663
Epoch 470 / 500 | iteration 10 / 30 | Total Loss: 4.736935615539551 | KNN Loss: 3.716346263885498 | BCE Loss: 1.0205895900726318
Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 4.670272350311279 | KNN Loss: 3.653196096420288 | BCE Loss: 1.0170761346817017
Epoch 470 / 500 | iteration 20 / 30 | Total Loss: 4.682635307312012 | KNN Loss: 3.6709365844726562 | BCE Loss: 1.0116984844207764
Epoch 470 / 500 | iteration 25 / 30 | Total Loss: 4.701333045959473 | KNN Loss: 3.6814544200897217 

Epoch 480 / 500 | iteration 10 / 30 | Total Loss: 4.723355770111084 | KNN Loss: 3.701094388961792 | BCE Loss: 1.0222612619400024
Epoch 480 / 500 | iteration 15 / 30 | Total Loss: 4.721074104309082 | KNN Loss: 3.7213540077209473 | BCE Loss: 0.9997198581695557
Epoch 480 / 500 | iteration 20 / 30 | Total Loss: 4.700201988220215 | KNN Loss: 3.6684844493865967 | BCE Loss: 1.0317176580429077
Epoch 480 / 500 | iteration 25 / 30 | Total Loss: 4.729432106018066 | KNN Loss: 3.6830976009368896 | BCE Loss: 1.0463342666625977
Epoch 481 / 500 | iteration 0 / 30 | Total Loss: 4.713353157043457 | KNN Loss: 3.698345899581909 | BCE Loss: 1.0150070190429688
Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 4.67999792098999 | KNN Loss: 3.65663743019104 | BCE Loss: 1.0233604907989502
Epoch 481 / 500 | iteration 10 / 30 | Total Loss: 4.685439109802246 | KNN Loss: 3.682257890701294 | BCE Loss: 1.0031814575195312
Epoch 481 / 500 | iteration 15 / 30 | Total Loss: 4.680056095123291 | KNN Loss: 3.6669938564300537

Epoch 491 / 500 | iteration 0 / 30 | Total Loss: 4.713675498962402 | KNN Loss: 3.686420202255249 | BCE Loss: 1.0272551774978638
Epoch 491 / 500 | iteration 5 / 30 | Total Loss: 4.739717483520508 | KNN Loss: 3.6963613033294678 | BCE Loss: 1.0433560609817505
Epoch 491 / 500 | iteration 10 / 30 | Total Loss: 4.723142147064209 | KNN Loss: 3.700502872467041 | BCE Loss: 1.022639274597168
Epoch 491 / 500 | iteration 15 / 30 | Total Loss: 4.720836639404297 | KNN Loss: 3.7010433673858643 | BCE Loss: 1.0197930335998535
Epoch 491 / 500 | iteration 20 / 30 | Total Loss: 4.700216770172119 | KNN Loss: 3.6801035404205322 | BCE Loss: 1.0201133489608765
Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 4.777024269104004 | KNN Loss: 3.728052854537964 | BCE Loss: 1.0489716529846191
Epoch 492 / 500 | iteration 0 / 30 | Total Loss: 4.656067848205566 | KNN Loss: 3.6576991081237793 | BCE Loss: 0.9983688592910767
Epoch 492 / 500 | iteration 5 / 30 | Total Loss: 4.736042022705078 | KNN Loss: 3.716486930847168 

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 1.9380,  2.3906,  3.0006,  2.2567,  3.8740,  0.6361,  3.0656,  2.5721,
          1.4370,  2.3509,  2.0332,  1.6066,  0.9440,  2.1109,  1.3601,  1.5618,
          2.6201,  2.0559,  1.9476,  2.5626,  1.7362,  2.5003,  2.7152,  3.0096,
          2.2980,  1.4191,  2.2821,  1.4492,  1.4496,  0.1748,  0.0623,  1.1730,
          0.3292,  0.9314,  1.1736,  1.4804,  1.3876,  3.7322,  0.4617,  1.4594,
          1.1210, -0.7043, -0.0831,  1.9769,  2.5086,  1.0815, -0.2059,  0.0990,
          1.7805,  1.4562,  2.1947,  0.0654,  1.6704,  0.6176, -0.3426,  1.1495,
          1.7938,  1.6236,  1.4176,  1.8316,  0.8881,  1.0403,  0.0438,  1.8676,
          1.6128,  2.0330, -1.9911,  0.4503,  2.6109,  2.5046,  2.0279,  0.5732,
          1.5381,  2.8802,  2.3922,  1.5608,  0.3403,  0.6326,  0.3242,  1.9697,
          0.1613,  0.6509,  2.1566, -0.2382,  0.5027, -0.9883, -2.2782, -0.1516,
          0.5367, -1.8180,  0.3130, -0.1467, -0.6027, -0.9736,  0.5923,  1.4121,
         -0.6954, -0.6870,  

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [12]:
dataset_ = [d[0].cpu() for d in dataset]

In [13]:
model = model.eval().cpu()
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 82.46it/s]


In [14]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [15]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [16]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [18]:
tensor_dataset = torch.stack(dataset_)

In [19]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [20]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [21]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [22]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [23]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [24]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [25]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [26]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [27]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [28]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [29]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [30]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [31]:
losses = []
accs = []
sparsity = []

In [32]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
Epoch: 00 | Batch: 000 / 026 | Total loss: 9.665 | Reg loss: 0.007 | Tree loss: 9.665 | Accuracy: 0.000000 | 0.09 sec/iter
Epoch: 00 | Batch: 001 / 026 | Total loss: 9.657 | Reg loss: 0.007 | Tree loss: 9.657 | Accuracy: 0.000000 | 0.084 sec/iter
Epoch: 00 | Batch: 002 / 026 | Total loss: 9.645 | Reg loss: 0.007 | Tree loss: 9.645 | Accuracy: 0.000000 | 0.078 sec/iter
Epoch: 00 | Batch: 003 / 026 | Total loss: 9.643 | Reg loss: 0.006 | Tree loss: 9.643 | Accuracy: 0.000000 | 0.076 sec/iter
Epoch: 00 | Batch: 004 / 026 | Total loss: 9.624 | Reg loss: 0.006 | Tree loss: 9.624 | Accuracy: 0.000000 | 0.078 sec/iter
Epoch: 00 | Batch: 005 / 026 | Total loss: 9.616 | Reg loss: 0.006 | Tree loss: 9.616 | Accuracy: 0.000000 | 0.078 sec/iter
Epoch: 00 | Batch: 006 / 026 | Total loss: 9.618 | Reg loss: 0.006 | Tree loss: 9.618 | Accuracy: 0.000000 | 0.077 sec/iter
Epoch: 00 | Batch: 007 / 026 | Total loss: 9

Epoch: 02 | Batch: 012 / 026 | Total loss: 9.292 | Reg loss: 0.007 | Tree loss: 9.292 | Accuracy: 0.080078 | 0.071 sec/iter
Epoch: 02 | Batch: 013 / 026 | Total loss: 9.272 | Reg loss: 0.007 | Tree loss: 9.272 | Accuracy: 0.099609 | 0.071 sec/iter
Epoch: 02 | Batch: 014 / 026 | Total loss: 9.268 | Reg loss: 0.007 | Tree loss: 9.268 | Accuracy: 0.095703 | 0.071 sec/iter
Epoch: 02 | Batch: 015 / 026 | Total loss: 9.272 | Reg loss: 0.008 | Tree loss: 9.272 | Accuracy: 0.076172 | 0.072 sec/iter
Epoch: 02 | Batch: 016 / 026 | Total loss: 9.258 | Reg loss: 0.008 | Tree loss: 9.258 | Accuracy: 0.085938 | 0.071 sec/iter
Epoch: 02 | Batch: 017 / 026 | Total loss: 9.250 | Reg loss: 0.008 | Tree loss: 9.250 | Accuracy: 0.078125 | 0.071 sec/iter
Epoch: 02 | Batch: 018 / 026 | Total loss: 9.244 | Reg loss: 0.008 | Tree loss: 9.244 | Accuracy: 0.095703 | 0.071 sec/iter
Epoch: 02 | Batch: 019 / 026 | Total loss: 9.235 | Reg loss: 0.009 | Tree loss: 9.235 | Accuracy: 0.074219 | 0.071 sec/iter
Epoch: 0

Epoch: 05 | Batch: 000 / 026 | Total loss: 8.989 | Reg loss: 0.010 | Tree loss: 8.989 | Accuracy: 0.089844 | 0.072 sec/iter
Epoch: 05 | Batch: 001 / 026 | Total loss: 8.981 | Reg loss: 0.010 | Tree loss: 8.981 | Accuracy: 0.066406 | 0.072 sec/iter
Epoch: 05 | Batch: 002 / 026 | Total loss: 8.971 | Reg loss: 0.010 | Tree loss: 8.971 | Accuracy: 0.076172 | 0.072 sec/iter
Epoch: 05 | Batch: 003 / 026 | Total loss: 8.956 | Reg loss: 0.010 | Tree loss: 8.956 | Accuracy: 0.082031 | 0.072 sec/iter
Epoch: 05 | Batch: 004 / 026 | Total loss: 8.950 | Reg loss: 0.010 | Tree loss: 8.950 | Accuracy: 0.072266 | 0.072 sec/iter
Epoch: 05 | Batch: 005 / 026 | Total loss: 8.939 | Reg loss: 0.011 | Tree loss: 8.939 | Accuracy: 0.076172 | 0.072 sec/iter
Epoch: 05 | Batch: 006 / 026 | Total loss: 8.941 | Reg loss: 0.011 | Tree loss: 8.941 | Accuracy: 0.054688 | 0.072 sec/iter
Epoch: 05 | Batch: 007 / 026 | Total loss: 8.918 | Reg loss: 0.011 | Tree loss: 8.918 | Accuracy: 0.097656 | 0.072 sec/iter
Epoch: 0

Epoch: 07 | Batch: 015 / 026 | Total loss: 8.531 | Reg loss: 0.016 | Tree loss: 8.531 | Accuracy: 0.072266 | 0.072 sec/iter
Epoch: 07 | Batch: 016 / 026 | Total loss: 8.516 | Reg loss: 0.016 | Tree loss: 8.516 | Accuracy: 0.080078 | 0.072 sec/iter
Epoch: 07 | Batch: 017 / 026 | Total loss: 8.517 | Reg loss: 0.016 | Tree loss: 8.517 | Accuracy: 0.074219 | 0.072 sec/iter
Epoch: 07 | Batch: 018 / 026 | Total loss: 8.493 | Reg loss: 0.017 | Tree loss: 8.493 | Accuracy: 0.109375 | 0.072 sec/iter
Epoch: 07 | Batch: 019 / 026 | Total loss: 8.482 | Reg loss: 0.017 | Tree loss: 8.482 | Accuracy: 0.095703 | 0.072 sec/iter
Epoch: 07 | Batch: 020 / 026 | Total loss: 8.477 | Reg loss: 0.017 | Tree loss: 8.477 | Accuracy: 0.087891 | 0.072 sec/iter
Epoch: 07 | Batch: 021 / 026 | Total loss: 8.451 | Reg loss: 0.017 | Tree loss: 8.451 | Accuracy: 0.103516 | 0.072 sec/iter
Epoch: 07 | Batch: 022 / 026 | Total loss: 8.436 | Reg loss: 0.018 | Tree loss: 8.436 | Accuracy: 0.128906 | 0.072 sec/iter
Epoch: 0

Epoch: 10 | Batch: 000 / 026 | Total loss: 8.239 | Reg loss: 0.018 | Tree loss: 8.239 | Accuracy: 0.056641 | 0.072 sec/iter
Epoch: 10 | Batch: 001 / 026 | Total loss: 8.212 | Reg loss: 0.018 | Tree loss: 8.212 | Accuracy: 0.089844 | 0.072 sec/iter
Epoch: 10 | Batch: 002 / 026 | Total loss: 8.197 | Reg loss: 0.018 | Tree loss: 8.197 | Accuracy: 0.078125 | 0.072 sec/iter
Epoch: 10 | Batch: 003 / 026 | Total loss: 8.194 | Reg loss: 0.018 | Tree loss: 8.194 | Accuracy: 0.080078 | 0.072 sec/iter
Epoch: 10 | Batch: 004 / 026 | Total loss: 8.151 | Reg loss: 0.018 | Tree loss: 8.151 | Accuracy: 0.085938 | 0.072 sec/iter
Epoch: 10 | Batch: 005 / 026 | Total loss: 8.161 | Reg loss: 0.018 | Tree loss: 8.161 | Accuracy: 0.068359 | 0.072 sec/iter
Epoch: 10 | Batch: 006 / 026 | Total loss: 8.147 | Reg loss: 0.018 | Tree loss: 8.147 | Accuracy: 0.070312 | 0.073 sec/iter
Epoch: 10 | Batch: 007 / 026 | Total loss: 8.125 | Reg loss: 0.018 | Tree loss: 8.125 | Accuracy: 0.078125 | 0.073 sec/iter
Epoch: 1

...

Epoch: 89 | Batch: 012 / 026 | Total loss: 2.798 | Reg loss: 0.030 | Tree loss: 2.798 | Accuracy: 0.115234 | 0.062 sec/iter
Epoch: 89 | Batch: 013 / 026 | Total loss: 2.781 | Reg loss: 0.030 | Tree loss: 2.781 | Accuracy: 0.125000 | 0.062 sec/iter
Epoch: 89 | Batch: 014 / 026 | Total loss: 2.815 | Reg loss: 0.030 | Tree loss: 2.815 | Accuracy: 0.087891 | 0.062 sec/iter
Epoch: 89 | Batch: 015 / 026 | Total loss: 2.806 | Reg loss: 0.030 | Tree loss: 2.806 | Accuracy: 0.128906 | 0.062 sec/iter
Epoch: 89 | Batch: 016 / 026 | Total loss: 2.781 | Reg loss: 0.030 | Tree loss: 2.781 | Accuracy: 0.130859 | 0.062 sec/iter
Epoch: 89 | Batch: 017 / 026 | Total loss: 2.773 | Reg loss: 0.030 | Tree loss: 2.773 | Accuracy: 0.119141 | 0.062 sec/iter
Epoch: 89 | Batch: 018 / 026 | Total loss: 2.776 | Reg loss: 0.030 | Tree loss: 2.776 | Accuracy: 0.111328 | 0.062 sec/iter
Epoch: 89 | Batch: 019 / 026 | Total loss: 2.768 | Reg loss: 0.030 | Tree loss: 2.768 | Accuracy: 0.119141 | 0.062 sec/iter
Epoch: 8

Epoch: 91 | Batch: 024 / 026 | Total loss: 2.739 | Reg loss: 0.030 | Tree loss: 2.739 | Accuracy: 0.105469 | 0.062 sec/iter
Epoch: 91 | Batch: 025 / 026 | Total loss: 2.759 | Reg loss: 0.030 | Tree loss: 2.759 | Accuracy: 0.114206 | 0.062 sec/iter
Average sparseness: 0.9821428571428567
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
Epoch: 92 | Batch: 000 / 026 | Total loss: 2.860 | Reg loss: 0.030 | Tree loss: 2.860 | Accuracy: 0.113281 | 0.062 sec/iter
Epoch: 92 | Batch: 001 / 026 | Total loss: 2.887 | Reg loss: 0.030 | Tree loss: 2.887 | Accuracy: 0.109375 | 0.062 sec/iter
Epoch: 92 | Batch: 002 / 026 | Total loss: 2.866 | Reg loss: 0.030 | Tree loss: 2.866 | Accuracy: 0.117188 | 0.062 sec/iter
Epoch: 92 | Batch: 003 / 026 | Total loss: 2.864 | Reg loss: 0.030 | Tree loss: 2.864 | Accuracy: 0.115234 | 0.062 sec/iter
Epoch: 92 | Batch: 004 / 026 | Total loss: 2.848 | Reg loss: 0.030 | Tree los

Epoch: 94 | Batch: 008 / 026 | Total loss: 2.822 | Reg loss: 0.030 | Tree loss: 2.822 | Accuracy: 0.113281 | 0.062 sec/iter
Epoch: 94 | Batch: 009 / 026 | Total loss: 2.834 | Reg loss: 0.030 | Tree loss: 2.834 | Accuracy: 0.113281 | 0.062 sec/iter
Epoch: 94 | Batch: 010 / 026 | Total loss: 2.846 | Reg loss: 0.030 | Tree loss: 2.846 | Accuracy: 0.091797 | 0.062 sec/iter
Epoch: 94 | Batch: 011 / 026 | Total loss: 2.802 | Reg loss: 0.030 | Tree loss: 2.802 | Accuracy: 0.105469 | 0.062 sec/iter
Epoch: 94 | Batch: 012 / 026 | Total loss: 2.798 | Reg loss: 0.030 | Tree loss: 2.798 | Accuracy: 0.121094 | 0.062 sec/iter
Epoch: 94 | Batch: 013 / 026 | Total loss: 2.815 | Reg loss: 0.030 | Tree loss: 2.815 | Accuracy: 0.134766 | 0.062 sec/iter
Epoch: 94 | Batch: 014 / 026 | Total loss: 2.782 | Reg loss: 0.030 | Tree loss: 2.782 | Accuracy: 0.119141 | 0.062 sec/iter
Epoch: 94 | Batch: 015 / 026 | Total loss: 2.802 | Reg loss: 0.030 | Tree loss: 2.802 | Accuracy: 0.103516 | 0.062 sec/iter
Epoch: 9

Epoch: 96 | Batch: 020 / 026 | Total loss: 2.780 | Reg loss: 0.030 | Tree loss: 2.780 | Accuracy: 0.080078 | 0.062 sec/iter
Epoch: 96 | Batch: 021 / 026 | Total loss: 2.761 | Reg loss: 0.030 | Tree loss: 2.761 | Accuracy: 0.111328 | 0.062 sec/iter
Epoch: 96 | Batch: 022 / 026 | Total loss: 2.787 | Reg loss: 0.030 | Tree loss: 2.787 | Accuracy: 0.115234 | 0.062 sec/iter
Epoch: 96 | Batch: 023 / 026 | Total loss: 2.765 | Reg loss: 0.030 | Tree loss: 2.765 | Accuracy: 0.109375 | 0.062 sec/iter
Epoch: 96 | Batch: 024 / 026 | Total loss: 2.738 | Reg loss: 0.030 | Tree loss: 2.738 | Accuracy: 0.132812 | 0.062 sec/iter
Epoch: 96 | Batch: 025 / 026 | Total loss: 2.745 | Reg loss: 0.030 | Tree loss: 2.745 | Accuracy: 0.144847 | 0.062 sec/iter
Average sparseness: 0.9821428571428567
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
Epoch: 97 | Batch: 000 / 026 | Total loss: 2.866 | Reg loss: 0.030 | Tree los

Epoch: 99 | Batch: 005 / 026 | Total loss: 2.808 | Reg loss: 0.030 | Tree loss: 2.808 | Accuracy: 0.142578 | 0.062 sec/iter
Epoch: 99 | Batch: 006 / 026 | Total loss: 2.850 | Reg loss: 0.030 | Tree loss: 2.850 | Accuracy: 0.103516 | 0.062 sec/iter
Epoch: 99 | Batch: 007 / 026 | Total loss: 2.834 | Reg loss: 0.030 | Tree loss: 2.834 | Accuracy: 0.097656 | 0.062 sec/iter
Epoch: 99 | Batch: 008 / 026 | Total loss: 2.827 | Reg loss: 0.030 | Tree loss: 2.827 | Accuracy: 0.103516 | 0.062 sec/iter
Epoch: 99 | Batch: 009 / 026 | Total loss: 2.814 | Reg loss: 0.030 | Tree loss: 2.814 | Accuracy: 0.113281 | 0.062 sec/iter
Epoch: 99 | Batch: 010 / 026 | Total loss: 2.837 | Reg loss: 0.030 | Tree loss: 2.837 | Accuracy: 0.091797 | 0.062 sec/iter
Epoch: 99 | Batch: 011 / 026 | Total loss: 2.818 | Reg loss: 0.030 | Tree loss: 2.818 | Accuracy: 0.089844 | 0.062 sec/iter
Epoch: 99 | Batch: 012 / 026 | Total loss: 2.781 | Reg loss: 0.030 | Tree loss: 2.781 | Accuracy: 0.099609 | 0.062 sec/iter
Epoch: 9

In [33]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [35]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 5.595238095238095


# Extract Rules

# Accumulate samples in the leaves

In [40]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 42


In [41]:
method = 'greedy'

In [42]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

# Tighten boundaries

In [44]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
9744
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Patte

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))
